## 빅데이터 실습

### 데이터포털 OPEN API 활용

#### 데이터 수집

##### OPEN API
- 한국관광공사 관광빅데이터 정보 서비스 GW 
    - 기본 URL >> http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList?
        - **`https` 아님!! >> `https`로 서비스 요청하면 `ssl` 관련 오류 발생**
    - 서비스키 >> Tg%2FJIztCN7ha9uY7fY2xfjH6FoMitSQmhaMQCP13WG8FcrxvH%2FJL6AN1XK6PzfBWd1Fyz2nGQAbpBmQebGy4yg%3D%3D
        - encoding 인증키
    - 파라미터(?뒤 옵션값) 
        - serviceKey == 본인키
        - numOfRows == 한페이지에 나타낼 결과 수 (basic = 10, 현재 URL에서는 60개 받음)
        - pageNo == 페이지번호 (basic = 1)
        - MobileOs == IOS, AND, WIN, **ETC**(기타)
        - MobileApp = **AppTest**
        - startYmd == 시작일자
        - endYmd == 종료일자

##### 추가 모듈
- XML을 딕셔너리로 한번 바꿔주는 모듈
- xmltodict
```shell
! pip3 install xmltodict
```

In [1]:
! pip3 install xmltodict

In [2]:
# 필요 라이브러리 등록
import requests
import json
import xmltodict
import pandas as pd
import datetime

In [3]:
# 웹으로 OPEN API 요청 기본 함수
def getRequestUrl(url) :
    request = requests(url)
    try :
        result = requests(request)
        if result.getcode() == 200 : # 웹 URL 요청결과 OK
            print(f'[{datetime.now}] URL 요청 성공!')
            return result.read().decode('UTF-8') # 웹으로 전달될때는 URL, UTF-8 인코딩, 돌려받을때는 UTF-8 디코딩
    except Exception as e :
        print(f'[{datetime.now}]getRequestUrl() 호출 예외 : {e}')
        return None

In [4]:
# 서비스키 등록
serviceKey = r'Tg%2FJIztCN7ha9uY7fY2xfjH6FoMitSQmhaMQCP13WG8FcrxvH%2FJL6AN1XK6PzfBWd1Fyz2nGQAbpBmQebGy4yg%3D%3D'

In [5]:
# 기준날짜
ymd = '20231201'

In [6]:
# 파라미터 변경하면서 OPEN API 요청할 URL 생성 및 결과 데이터 리턴함수
# 만약 시작일자, 종료일자가 다르다면 파라미터를 두개 받음, 받은 데이터는 시작일 == 종료일 이라 파라미터 하나
baseUrl = 'http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList?'
parameter = 'numOfRows=60&pageNo=1&MobileOS=ETC&MobileApp=AppTest&' + \
            f'serviceKey={serviceKey}&startYmd={ymd}&endYmd={ymd}'
totalUrl = baseUrl + parameter
totalUrl

'http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList?numOfRows=60&pageNo=1&MobileOS=ETC&MobileApp=AppTest&serviceKey=Tg%2FJIztCN7ha9uY7fY2xfjH6FoMitSQmhaMQCP13WG8FcrxvH%2FJL6AN1XK6PzfBWd1Fyz2nGQAbpBmQebGy4yg%3D%3D&startYmd=20231201&endYmd=20231201'

In [7]:
# 실행 테스트
# res.status_code >> 200(OK) or 404/500  ||  res.content
result = requests.get(totalUrl)
result.status_code
result.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header>\n  <resultCode>0000</resultCode>\n  <resultMsg>OK</resultMsg>\n</header><body><items>\n  <item>\n    <areaCode>11</areaCode>\n    <areaNm>서울특별시</areaNm>\n    <daywkDivCd>5</daywkDivCd>\n    <daywkDivNm>금요일</daywkDivNm>\n    <touDivCd>1</touDivCd>\n    <touDivNm>현지인(a)</touDivNm>\n    <touNum>4603042.5</touNum>\n    <baseYmd>20231201</baseYmd>\n  </item>\n  <item>\n    <areaCode>11</areaCode>\n    <areaNm>서울특별시</areaNm>\n    <daywkDivCd>5</daywkDivCd>\n    <daywkDivNm>금요일</daywkDivNm>\n    <touDivCd>2</touDivCd>\n    <touDivNm>외지인(b)</touDivNm>\n    <touNum>1448384.0</touNum>\n    <baseYmd>20231201</baseYmd>\n  </item>\n  <item>\n    <areaCode>11</areaCode>\n    <areaNm>서울특별시</areaNm>\n    <daywkDivCd>5</daywkDivCd>\n    <daywkDivNm>금요일</daywkDivNm>\n    <touDivCd>3</touDivCd>\n    <touDivNm>외국인(c)</touDivNm>\n    <touNum>30583.54999999999</touNum>\n    <baseYmd>20231201</baseYmd>\n  </item>\n  <item>\n    <areaC

In [8]:
## openAPI로 넘어온 text(xml타입)
## xmltodicy.parse() >> xml문서를 파이썬 dictionary 타입 변환
## json.dumps() dictionary를 json 타입으로 변환
dictTemp = xmltodict.parse(result.text)

In [9]:
dictTemp['response']['header']['resultCode'] # 0000 == OK

'0000'

In [10]:
# 해당 요소가 여러 아이템을 가지고 있다면, 이는 일반적으로 리스트 형태
type(dictTemp['response']['body']['items']['item'])

list

In [11]:
pd.DataFrame(dictTemp['response']['body']['items']['item'])

,areaCode,areaNm,daywkDivCd,daywkDivNm,touDivCd,touDivNm,touNum,baseYmd
0,11,서울특별시,5,금요일,1,현지인(a),4603042.5,20231201
1,11,서울특별시,5,금요일,2,외지인(b),1448384.0,20231201
2,11,서울특별시,5,금요일,3,외국인(c),30583.54999999999,20231201
3,26,부산광역시,5,금요일,1,현지인(a),1583729.0,20231201
4,26,부산광역시,5,금요일,2,외지인(b),323697.5,20231201
5,26,부산광역시,5,금요일,3,외국인(c),8219.360000000004,20231201
6,27,대구광역시,5,금요일,1,현지인(a),1206880.5,20231201
7,27,대구광역시,5,금요일,2,외지인(b),238301.0,20231201
8,27,대구광역시,5,금요일,3,외국인(c),1004.25,20231201
9,28,인천광역시,5,금요일,1,현지인(a),1364107.5,20231201


###### 단일테스트 완료 / 함수 생성
- getRequsetUrl()
- getKorTouristData()
- 2023-12-01 ~ 20203-12-31 반복문

In [12]:
# 웹으로 OPEN API URL 호출 함수
def getRequestUrl(url) :
    try :
        result = requests.get(url, verify=False)
        # 요청결과 OK
        if result.status_code == 200 : 
            return result.text
    except Exception as e:
        print(f'getRequestURL() 함수호출 예외 : {e}')
        return None

In [13]:
# OPEN API 호출 함수
def getKorTouristData(ymd) :
    serviceKey = r'Tg%2FJIztCN7ha9uY7fY2xfjH6FoMitSQmhaMQCP13WG8FcrxvH%2FJL6AN1XK6PzfBWd1Fyz2nGQAbpBmQebGy4yg%3D%3D'
    baseUrl = 'http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList?'
    parameter = 'numOfRows=60&pageNo=1&MobileOS=ETC&MobileApp=AppTest&' + \
                f'serviceKey={serviceKey}&startYmd={ymd}&endYmd={ymd}'
    totalUrl = baseUrl + parameter

    result = getRequestUrl(totalUrl)
    if result == None :
        return None
    else :
        return result

In [14]:
# 실행 테스트
data = getKorTouristData('20231202')

In [15]:
# xml을 딕셔너리로 형변환 후 필요한 데이터까지만 추출
xmltodict.parse(data)['response']['body']['items']['item']

[{'areaCode': '11',
  'areaNm': '서울특별시',
  'daywkDivCd': '6',
  'daywkDivNm': '토요일',
  'touDivCd': '1',
  'touDivNm': '현지인(a)',
  'touNum': '4085018.0',
  'baseYmd': '20231202'},
 {'areaCode': '11',
  'areaNm': '서울특별시',
  'daywkDivCd': '6',
  'daywkDivNm': '토요일',
  'touDivCd': '2',
  'touDivNm': '외지인(b)',
  'touNum': '1940487.5',
  'baseYmd': '20231202'},
 {'areaCode': '11',
  'areaNm': '서울특별시',
  'daywkDivCd': '6',
  'daywkDivNm': '토요일',
  'touDivCd': '3',
  'touDivNm': '외국인(c)',
  'touNum': '31302.643199999977',
  'baseYmd': '20231202'},
 {'areaCode': '26',
  'areaNm': '부산광역시',
  'daywkDivCd': '6',
  'daywkDivNm': '토요일',
  'touDivCd': '1',
  'touDivNm': '현지인(a)',
  'touNum': '1479635.5',
  'baseYmd': '20231202'},
 {'areaCode': '26',
  'areaNm': '부산광역시',
  'daywkDivCd': '6',
  'daywkDivNm': '토요일',
  'touDivCd': '2',
  'touDivNm': '외지인(b)',
  'touNum': '484620.5',
  'baseYmd': '20231202'},
 {'areaCode': '26',
  'areaNm': '부산광역시',
  'daywkDivCd': '6',
  'daywkDivNm': '토요일',
  'touDivCd'

In [16]:
# 추가 라이브러리
from tqdm import tqdm
import time

In [17]:
## 반복문으로 처리
korTouristData = []
year = 2023
month = 12

for day in tqdm(range(1, 32)):
    requsetYmd = f'{year}{month}{day:02d}'
    data = getKorTouristData(requsetYmd)
    # 해당 날짜에 데이터가 없을수 있음
    if xmltodict.parse(data)['response']['body']['items'] == None : 
        resultList = []
    else :
        resultList = xmltodict.parse(data)['response']['body']['items']['item']

    korTouristData = korTouristData + resultList

100%|██████████| 31/31 [01:22<00:00,  2.66s/it]


In [18]:
len(korTouristData)

1530

In [20]:
pd.DataFrame(korTouristData)

,areaCode,areaNm,daywkDivCd,daywkDivNm,touDivCd,touDivNm,touNum,baseYmd
0,11,서울특별시,5,금요일,1,현지인(a),4603042.5,20231201
1,11,서울특별시,5,금요일,2,외지인(b),1448384.0,20231201
2,11,서울특별시,5,금요일,3,외국인(c),30583.54999999999,20231201
3,26,부산광역시,5,금요일,1,현지인(a),1583729.0,20231201
4,26,부산광역시,5,금요일,2,외지인(b),323697.5,20231201
...,...,...,...,...,...,...,...,...
1525,51,강원특별자치도,7,일요일,2,외지인(b),694498.0,20231231
1526,51,강원특별자치도,7,일요일,3,외국인(c),2551.5179999999996,20231231
1527,52,전북특별자치도,7,일요일,1,현지인(a),910780.5,20231231
1528,52,전북특별자치도,7,일요일,2,외지인(b),408299.5,20231231


In [22]:
dfKorTouristData = pd.DataFrame(korTouristData)

In [24]:
dfKorTouristData.tail()

,areaCode,areaNm,daywkDivCd,daywkDivNm,touDivCd,touDivNm,touNum,baseYmd
1525,51,강원특별자치도,7,일요일,2,외지인(b),694498.0,20231231
1526,51,강원특별자치도,7,일요일,3,외국인(c),2551.5179999999996,20231231
1527,52,전북특별자치도,7,일요일,1,현지인(a),910780.5,20231231
1528,52,전북특별자치도,7,일요일,2,외지인(b),408299.5,20231231
1529,52,전북특별자치도,7,일요일,3,외국인(c),1043.7619000000004,20231231


In [26]:
dfKorTouristData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1530 entries, 0 to 1529
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   areaCode    1530 non-null   object
 1   areaNm      1530 non-null   object
 2   daywkDivCd  1530 non-null   object
 3   daywkDivNm  1530 non-null   object
 4   touDivCd    1530 non-null   object
 5   touDivNm    1530 non-null   object
 6   touNum      1530 non-null   object
 7   baseYmd     1530 non-null   object
dtypes: object(8)
memory usage: 95.8+ KB


In [27]:
# 엑셀파일로 저장
dfKorTouristData.to_excel('./data/Tourist_dataPortal.xlsx', index=False)